In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-25T08:49:10.562373+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [3]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')

In [4]:
# Get the token from Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVELOOP_ORG_ID

In [6]:
import langchain
import tqdm
import requests
import typing
%watermark --iversions 

langchain: 0.0.336
tqdm     : 4.66.1
requests : 2.31.0



In [5]:
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PagedPDFSplitter
import requests
import tqdm
from typing import List

In [9]:
from langchain.document_loaders import (
    CSVLoader,
    DirectoryLoader,
    GitLoader,
    NotebookLoader,
    OnlinePDFLoader,
    PythonLoader,
    TextLoader,
    UnstructuredFileLoader,
    UnstructuredHTMLLoader,
    UnstructuredPDFLoader,
    UnstructuredWordDocumentLoader,
    WebBaseLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
data_source=r'C:\Users\jfrancis\Downloads\SQL Interview qns.pdf'
loader = UnstructuredPDFLoader(data_source)
# Chunk size is a major trade-off parameter to control result accuracy over computaion
CHUNK_SIZE = 1000
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)
docs

[Document(page_content="SQL INTERVIEW QUESTIONS AND ANSWERS\n\nSQL INTERVIEW QUESTIONS & ANSWERS\n\n1 | P a g e\n\nSQL INTERVIEW QUESTIONS AND ANSWERS\n\nMOST FREQUENTLY ASKED SQL INTERVIEW QUESTIONS\n\n1. What is the difference between “Stored Procedure” and “Function”?\n\n1. A procedure can have both input and output parameters, but a function can only\n\nhave input parameters.\n\n2. Inside a procedure we can use DML (INSERT/UPDATE/DELETE) statements. But\n\ninside a function we can't use DML statements.\n\n3. We can't utilize a Stored Procedure in a Select statement. But we can use a function\n\nin a Select statement.\n\n4. We can use a Try-Catch Block in a Stored Procedure but inside a function we can't\n\nuse a Try-Catch block.\n\n5. We can use transaction management in a procedure but we can't in a function. 6. We can't join a Stored Procedure but we can join functions. 7. Stored Procedures cannot be used in the SQL statements anywhere in the WHERE/HAVING/SELECT section. But we c

Now use a open source hugging face embedding as the openAI one could be costly (due to the volume of data)

In [14]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Load it to active loop vector store

In [15]:
my_activeloop_dataset_name = "sql_interview"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embeddings)
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating 117 embeddings in 1 batches of size 117:: 100%|████████████████████████████████| 1/1 [01:42<00:00, 102.36s/it]

Dataset(path='hub://jfrancis/sql_interview', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (117, 1)     str     None   
 metadata     json      (117, 1)     str     None   
 embedding  embedding  (117, 768)  float32   None   
    id        text      (117, 1)     str     None   


['85a38147-8b50-11ee-b60b-401c83da435e',
 '85a38148-8b50-11ee-98a7-401c83da435e',
 '85a38149-8b50-11ee-b082-401c83da435e',
 '85a3814a-8b50-11ee-b8f9-401c83da435e',
 '85a3814b-8b50-11ee-9aea-401c83da435e',
 '85a3814c-8b50-11ee-8297-401c83da435e',
 '85a3814d-8b50-11ee-9092-401c83da435e',
 '85a3814e-8b50-11ee-bdca-401c83da435e',
 '85a3814f-8b50-11ee-a0b8-401c83da435e',
 '85a38150-8b50-11ee-9f19-401c83da435e',
 '85a38151-8b50-11ee-99ea-401c83da435e',
 '85a38152-8b50-11ee-bac4-401c83da435e',
 '85a38153-8b50-11ee-b229-401c83da435e',
 '85a38154-8b50-11ee-b51c-401c83da435e',
 '85a38155-8b50-11ee-9369-401c83da435e',
 '85a38156-8b50-11ee-bdf0-401c83da435e',
 '85a38157-8b50-11ee-9e80-401c83da435e',
 '85a38158-8b50-11ee-96ea-401c83da435e',
 '85a38159-8b50-11ee-a556-401c83da435e',
 '85a3815a-8b50-11ee-b910-401c83da435e',
 '85a3815b-8b50-11ee-ba1e-401c83da435e',
 '85a3815c-8b50-11ee-8dfa-401c83da435e',
 '85a3815d-8b50-11ee-a04c-401c83da435e',
 '85a3815e-8b50-11ee-88e9-401c83da435e',
 '85a3815f-8b50-

Since the dataset is stored on Activeloop, you can load it later without recomputing embeddings. This is a significant benefit cause it would save you time and computational resources. LangChain has a wrapper around Deep Lake, allowing you to use it as a Vector Store.

In [9]:
#db = DeepLake(dataset_path="hub://jfrancis/sql_interview", read_only=True, embedding=embeddings)

### Creating the retrieval tool

Now, we’ll construct the custom tool function that will retrieve the relevant documents from the Deep Lake database:

In [16]:
# Get the retriever object from the deep lake db object
retriever = db.as_retriever()

In [17]:
from langchain.agents import tool

# We define some variables that will be used inside our custom tool
# We're creating a custom tool that looks for relevant documents in our deep lake db
CUSTOM_TOOL_N_DOCS = 3 # number of retrieved docs from deep lake to consider
CUSTOM_TOOL_DOCS_SEPARATOR ="\n\n" # how to join together the retrieved docs to form a single string

# We use the tool decorator to wrap a function that will become our custom tool
# Note that the tool has a single string as input and returns a single string as output
# The name of the function will be the name of our custom tool
# The docstring of the function will be the description of our custom tool
# The description is used by the agent to decide whether to use the tool for a specific query
@tool
def retrieve_n_docs_tool(query: str) -> str:
    """ Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)[:CUSTOM_TOOL_N_DOCS]
    texts = [doc.page_content for doc in docs]
    texts_merged = CUSTOM_TOOL_DOCS_SEPARATOR.join(texts)
    return texts_merged

Our function, retrieve_n_docs_tool, is designed with a specific purpose in mind - to search for and retrieve relevant documents based on a given query. It accepts a single string input, which is the user's question or query, and it's designed to return a single string output.

To find the relevant documents, our function makes use of the retriever object's get_relevant_documents method. Given the query, this method searches for and returns a list of the most relevant documents. But we only need some of the documents it finds. We only want the top few. That's where the [: CUSTOM_TOOL_N_DOCS] slice comes in. It allows us to select the top CUSTOM_TOOL_N_DOCS number of documents from the list, where CUSTOM_TOOL_N_DOCS is a predefined constant that tells us how many documents to consider. In this case, that’s 3 documents, as we specified. 

Now that we have our top documents, we want to extract the text from each of them. We achieve this using a list comprehension that iterates over each document in our list, docs, and extracts the page_content or text from each document. The result is a list of the top 3 relevant document texts. Next, we want to join these individual texts from a list into a single string using .join(texts) method. Finally, our function returns texts_merged, a single string that comprises the joined texts from the relevant documents. The @tool decorator wraps the function, turning it into a custom tool. 

### Using the tool with an agent

 We can now initialize the agent that uses our custom tool.

In [18]:
# Load a LLM to create an agent using our custom tool
from langchain.llms import OpenAI
# Classes for initializing the agent that will use the custom tool
from langchain.agents import initialize_agent, AgentType

# Let's create an agent that uses our custom tool
# We set verbose=True to check if the agent is using the tool for generating the final answer
llm = OpenAI(model="text-davinci-003", temperature=0)
agent = initialize_agent([retrieve_n_docs_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

The initialize_agent function takes in three parameters: a list of the custom tools, the language learning model, and the type of agent. We're using the OpenAI LLM and specifying the agent type as ZERO_SHOT_REACT_DESCRIPTION. With verbose=True we can check if the agent is using the tool when generating the final answer.

Once the agent has been set up, it can be queried:

In [19]:
response = agent.run("What is a primary key?")



> Entering new AgentExecutor chain...
 A primary key is a unique identifier for a record in a database table.
Action: retrieve_n_docs_tool
Action Input: "What is a primary key?"
Observation: Frequently asked SQL Interview Questions A primary key is a column whose values uniquely identify every row in a table. Primary key values can never be reused. If a row is deleted from the table, its primary key may not be assigned to any new rows in the future. To define a field as primary key, following conditions had to be met :

1. No two rows can have the same primary key value. 2. Every row must have a primary key value 3. Primary key field cannot be null 4. Values in primary key columns can never be modified or updated

10 | P a g e

SQL INTERVIEW QUESTIONS AND ANSWERS

36.What is a Composite Key ?

A Composite primary key is a type of candidate key, which represents a set of columns whose values uniquely identify every row in a table.

For example - if "Employee_ID" and "Employee Name" in

By reading the agent printed output, we see that the agent decided to use the retrieve_n_docs_tool tool to retrieve relevant documents to the SQL Interviews query. The final answer is then generated using the information contained in the retrieved documents.

In [20]:
print(response)

A primary key is a column or set of columns whose values uniquely identify every row in a table. Primary key values can never be reused, and they cannot be null. A composite key is a type of candidate key which represents a set of columns whose values uniquely identify every row in a table. A foreign key is a field or collection of fields in one table that uniquely identifies a row of another table.


#### Just trying out the Opensource alternative for openai text generaiton with huggingface

In [27]:
from langchain import HuggingFaceHub
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature":0.8, "max_length":512}
)

In [28]:
qa = RetrievalQA.from_chain_type(llm=flan_t5, chain_type='stuff', retriever=db.as_retriever())

In [29]:
qa.run("What is a primary key?")

'A primary key is a column whose values uniquely identify every row in a table.'